# Diabetic_Retinopathy Using AlexNet Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from sklearn.utils import shuffle
from skimage.transform import resize
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
import glob
import torch
import torchvision.models as models

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
alexnet = models.alexnet(pretrained = True)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
Healthy = glob.glob('Diabetic/Healthy/*.png')
Mild_DR = glob.glob('Diabetic/Mild DR/*.png')
Moderate_DR = glob.glob('Diabetic/Moderate DR/*.png')
Proliferate_DR = glob.glob('Diabetic/Proliferate DR/*.png')
Severe_DR = glob.glob('Diabetic/Severe DR/*.png')

In [5]:
image = []
label = []

for i in Healthy:
    img = plt.imread(i)
    image.append(img)
    label.append(0)
    
for i in Mild_DR:
    img = plt.imread(i)
    image.append(img)
    label.append(1)
    
for i in Moderate_DR:
    img = plt.imread(i)
    image.append(img)
    label.append(2)
    
for i in Proliferate_DR:
    img = plt.imread(i)
    image.append(img)
    label.append(3)
    
for i in Severe_DR:
    img = plt.imread(i)
    image.append(img)
    label.append(4)

In [6]:
image, label = shuffle(image, label)

In [7]:
X = []
for i in image:
    X.append(resize(i, (227,227,3)))
X = np.array(X)

In [8]:
y = np.array(label)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [11]:
X_train.shape

(2200, 227, 227, 3)

In [12]:
y_train.shape

(2200,)

In [13]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [14]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [16]:
model.fit(X_train,y_train,
          epochs = 50,
          validation_split = 0.1)

Epoch 1/50
62/62 [==============================] - 82s 1s/step - loss: 12.1934 - accuracy: 0.4768 - val_loss: 3.2938 - val_accuracy: 0.2636
Epoch 2/50
62/62 [==============================] - 81s 1s/step - loss: 1.6572 - accuracy: 0.5258 - val_loss: 1.7273 - val_accuracy: 0.2636
Epoch 3/50
62/62 [==============================] - 80s 1s/step - loss: 1.2085 - accuracy: 0.5768 - val_loss: 3.8453 - val_accuracy: 0.1318
Epoch 4/50
62/62 [==============================] - 81s 1s/step - loss: 1.1885 - accuracy: 0.5808 - val_loss: 2.1055 - val_accuracy: 0.1318
Epoch 5/50
62/62 [==============================] - 81s 1s/step - loss: 1.1110 - accuracy: 0.5980 - val_loss: 197.4334 - val_accuracy: 0.1273
Epoch 6/50
62/62 [==============================] - 81s 1s/step - loss: 1.1074 - accuracy: 0.5975 - val_loss: 82.7348 - val_accuracy: 0.1773
Epoch 7/50
62/62 [==============================] - 81s 1s/step - loss: 1.3112 - accuracy: 0.5848 - val_loss: 102.1166 - val_accuracy: 0.4045
Epoch 8/50
62/

In [17]:
model.evaluate(X_train, y_train)

69/69 [==============================] - 19s 269ms/step - loss: 2758.9136 - accuracy: 0.3627


[2758.91357421875, 0.3627272844314575]